In [6]:
import pandas as pd
import requests
import json

url_nodes = "https://api.arrayofthings.org/api/nodes"
url_obser = "https://api.arrayofthings.org/api/observations"
url_sensor = "https://api.arrayofthings.org/api/sensors"



def sensors_df():
    sensor_data = []
    resp = requests.get(url_sensor)
    sensors = resp.json()
    sensor_meta = sensors["meta"]
    sensors_data = sensors["data"]
    for data in sensors_data:
        sensor_dict = { "path": data["path"],
                       "min_value": data["min"],
                       "max_value": data["max"],
                       "uom": data["uom"]}
        sensor_data.append(sensor_dict)
    print("Information about sensor_df")
    sensor_df = pd.DataFrame(sensor_data)
    return sensor_df
    

def node_df():
    node_data = []
    resp = requests.get(url_nodes)
    nodes = resp.json()
    nodes_meta = nodes["meta"]
    nodes_data = nodes["data"]
    for data in nodes_data:
        location = data["location"]["geometry"]["coordinates"]
        node_dict = {"EPSG:4326_Long": location[0],
                     "EPSG:4326_Lat": location[1],
                   "Address": data["address"],
                   'Description': data['description']}
        node_data.append(node_dict)  
    print("Information about available AOT nodes:")
    node_df = pd.DataFrame(node_data)
    return node_df


def observation_df():
    observation_data = []
    resp = requests.get(url_obser, stream= True)
    observ = resp.json()
    observ_meta = observ["meta"]
    observ_data = observ["data"]
    for data in observ_data:
        location = data["location"]["geometry"]["coordinates"]
        observ_dict = {"EPSG:4326_Long": location[0],
                     "EPSG:4326_Lat": location[1],
                     "Measurement": data["value"],
                     'Uom': data['uom'],
                     'Timestamp':data['timestamp'],
                     "Sensor_path":data["sensor_path"],
                     "Node_version":data["node_vsn"]
                     
                        }
        observation_data.append(observ_dict)  
    print(" AOT obervation data:")
    observ_df = pd.DataFrame(observation_data)
    return observ_df

In [7]:
observation_df().to_csv("./master_df.csv")

 AOT obervation data:


In [12]:
node_df().to_csv('./chicago_nodes.csv')

Information about available AOT nodes:


In [13]:
pwd

'/home/arjun/Array_of_things/leaflet_map/src/data/node_data'

In [8]:
import os
import schedule
import time
import functools
from datetime import date


def with_logging(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('LOG: Running job "%s"' % func.__name__)
        result = func(*args, **kwargs)
        print('LOG: Job "%s" completed' % func.__name__)
        return result
    return wrapper

@with_logging
def create_master_df():
    master_df = pd.read_csv('./master_df.csv')
    observ_df = observation_df()
    master_df = master_df.append(observ_df)
    if os.path.exists("./master_df.csv"):
        os.remove("./master_df.csv")
    master_df.to_csv('./master_df.csv')
 
@with_logging
def create_data_backup(DATA_BACK_UP_TIME):
    if os.path.exists("./master_df.csv"):
        back_up_file = pd.read_csv("./master_df.csv")
        back_up_file.to_csv(f"{date.today()}_master_df.csv")
        
    

In [10]:

schedule.every(5).minutes.do(create_master_df)
#schedule.every().day.at("5:00").do(create_data_backup)
print("Schedule Started")

#while True:
#    schedule.run_pending()
#    time.sleep(1)

Schedule Started
